# TP Implémentation du modèle linéaire L1 régularisé


Puisqu'il y a du code fourni pour une application similaire je commence par l'éxécuter pour voir ce qu'il fait et ce qui sera réutilisable.

In [178]:
import numpy as np
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.svm import SVC
from sklearn import cross_validation
class RandomOuSvmClassifier(BaseEstimator,ClassifierMixin):
    """ si la nature du classifier est random il predit au hasard des 0 et
    des 1 sinon il utilise un svm pour predire"""
    def __init__(self,nature="random"):
        self.nature=nature
        self.svm=SVC()
    def fit(self, X, y):
        if(self.nature=="svm"):
            self.svm.fit(X, y)
        return self
    def predict(self, X):
        if(self.nature=="random"):
            return np.random.randint(0,2,len(X))
        else:
            return self.svm.predict(X)

C'est visiblement l'implémentation d'un classifieur avec la même interface que ceux de sklearn, et donc la même que celle du classifieur demandé
je jette un oeil aux docs de http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html et http://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html pour voir s'il est nécessaire d'en hériter.(la doc dit que oui mais je ne comprends pas trop pourquoi...)

In [179]:
from sklearn import datasets
from sklearn import metrics
iris = datasets.load_iris()
X = iris.data
y = iris.target
classifieurRandom= RandomOuSvmClassifier(nature="random")
classifieurSvm= RandomOuSvmClassifier(nature="svm")
scoresRandom = cross_validation.cross_val_score(classifieurRandom, X, y, cv=5
,scoring="accuracy")
scoresSvm = cross_validation.cross_val_score(classifieurSvm, X, y, cv=5
,scoring="accuracy")

In [182]:
scoresRandom

array([ 0.33333333,  0.4       ,  0.26666667,  0.26666667,  0.33333333])

In [183]:
scoresSvm

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

Ce deuxième bout de code m'a donc montré comment charger un ensemble de données qui possède 3 classes (au vu du random effectué dans le predict et de ses résultats) Il faudra donc en utiliser un autre car notre classifications se fait sur 2 classes. Il me montre aussi comment utiliser la cross-validation.

In [6]:
import numpy as np
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from numpy.linalg import norm

class L1LinearRegularisedClassifier(BaseEstimator,ClassifierMixin):
   
    def __init__(self,regularisationWeight =0.0, step = 0.0001,iterations = 5000):       
        
        self.regularisationWeight = regularisationWeight
        self.step = step
        self.theta = np.array(np.random.rand(X.shape[1]+1),dtype='float64')
        
        self.iterations = iterations
    
    def regularization(self,theta):
        return self.regularisationWeight * norm(theta,0)
    
    def fit(self, X, y):
        
        self.stochastic_gradient_descent_L1(X,y,self.theta,self.iterations,self.step)
        return self
    
    def stochastic_gradient_descent_L1(self,X,y,theta,I,step):
        print theta.shape
        print X.shape
        Xchanged = np.concatenate((np.ones((X.shape[0],1),dtype='float64'),X),axis=1)
        for it in range(I):
            idx = np.random.randint(y.size)
            gradCost = np.zeros(Xchanged.shape[1])
            prod = np.dot(Xchanged[idx],self.theta)
            gradCost = 2.0 * (y[idx]-prod)*Xchanged[idx]
            
            thetaPrime = self.theta + step*gradCost
            for i in range(self.theta.size):
                if self.theta[i]*thetaPrime[i] > 0 :
                    self.theta[i] = 0
                else :
                    self.theta[i] = thetaPrime[i]
            if it % 500 == 0:
                y_pred = self.predict(X)
                print it
                print self.cost_function(y,y_pred,mean_squared_error,self.regularization), "coût"
                print accuracy_score(y,y_pred), "accuracy"
        
    
    def cost_function(self,y,y_pred,error_function,regularisation_function) :
        cost = error_function(y,y_pred)-regularisation_function(self.theta)
        return cost
    
    def stochastic_gradient_L1(self,theta,X,y,idx):
        return 2.0 * (y-np.dot(theta,X))*X[idx]
    
    def linear_function(self,X,theta):
        return np.dot(X,theta)
    
    """def predict_with_theta(self, X, theta):
        
        y_pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            y_pred[i] = linear_function(X[i],theta)
            if y_pred[i] > 0:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
        return y_pred"""
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        Xchanged = np.concatenate((np.ones((X.shape[0],1),dtype='float64'),X),axis=1) 
        for i in range(Xchanged.shape[0]):
            y_pred[i] = self.linear_function(Xchanged[i],self.theta)
            if y_pred[i] > 0:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
        return y_pred

from sklearn import datasets
from sklearn import metrics
from sklearn.datasets import fetch_mldata
digits = datasets.load_digits(n_class = 2)
X = digits.data
y = digits.target
y = 2*y - 1
print type(X[0][0])
print type(y[0])
print y[:10]
classifieurL1= L1LinearRegularisedClassifier()
scoresL1 = cross_validation.cross_val_score(classifieurL1, X, y, cv=5,scoring="accuracy"
)


<type 'numpy.float64'>
<type 'numpy.int32'>
[-1  1 -1  1 -1  1 -1 -1  1  1]
(65L,)
(287L, 64L)
0
1.0 coût
0.0 accuracy
500
1.0 coût
0.0 accuracy
1000
1.0 coût
0.0 accuracy
1500
1.97909407666 coût
0.505226480836 accuracy
2000
1.0 coût
0.0 accuracy
2500
1.0 coût
0.0 accuracy
3000
1.0 coût
0.0 accuracy
3500
1.0 coût
0.0 accuracy
4000
1.97909407666 coût
0.505226480836 accuracy
4500
1.97909407666 coût
0.505226480836 accuracy
(65L,)
(287L, 64L)
0
1.0 coût
0.0 accuracy
500
1.0 coût
0.0 accuracy
1000
1.0 coût
0.0 accuracy
1500
1.0 coût
0.0 accuracy
2000
1.0 coût
0.0 accuracy
2500
1.0 coût
0.0 accuracy
3000
2.02090592334 coût
0.463414634146 accuracy
3500
1.97909407666 coût
0.505226480836 accuracy
4000
1.97909407666 coût
0.505226480836 accuracy
4500
1.97909407666 coût
0.505226480836 accuracy
(65L,)
(288L, 64L)
0
1.0 coût
0.0 accuracy
500
1.97222222222 coût
0.506944444444 accuracy
1000
1.0 coût
0.0 accuracy
1500
1.0 coût
0.0 accuracy
2000
1.0 coût
0.0 accuracy
2500
1.97222222222 coût
0.5069444444

[-1  1 -1  1 -1  1 -1 -1  1  1]


NameError: global name 'theta' is not defined

http://www.aclweb.org/anthology/P09-1054

In [34]:
np.random.randint(2)

0

In [37]:
Y = np.array([1,2,3,4,5])
Y

array([1, 2, 3, 4, 5])

In [39]:
Y.shape

(5L,)

In [40]:
Y.size


5

In [69]:
A = np.random.rand(6,5)

In [55]:
A

array([[ 0.61990672,  0.85698778,  0.30227644,  0.33878824,  0.99416707],
       [ 0.62365261,  0.58393254,  0.54403266,  0.3566358 ,  0.09184679],
       [ 0.85410289,  0.58495266,  0.94803139,  0.03989068,  0.29810894],
       [ 0.74813547,  0.85936373,  0.35325312,  0.02635477,  0.24077968],
       [ 0.18114573,  0.57853052,  0.06748894,  0.09179829,  0.12952997]])

In [61]:
sum(A[1])

2.2001003949167433

In [63]:
8%9


8

In [152]:
mean_squared_error(np.ones(5),np.zeros(5))

1.0